In [7]:
import pandas as pd
import numpy as np
import time

# Importing dataset

In [301]:
datasets = [
    "data/opsahl-ucsocial/out.opsahl-ucsocial",
    "data/soc-sign-bitcoinalpha/out.soc-sign-bitcoinalpha",
    "data/soc-sign-bitcoinotc/out.soc-sign-bitcoinotc",
    "data/digg-friends/out.digg-friends",
    "data/prosper-loans/out.prosper-loans"]
skiprows = [[0, 1], [0], [0], [0], [0]]
current = 0
graph = pd.read_csv(
    datasets[current],
    names=["_from", "_to", "_weight", "_timestamp"],
    sep=" |\t",
    engine ='python',
    skiprows=skiprows[current]
)
# print(graph.dtypes)
# print(graph.head())

In [302]:
V = np.unique(graph["_from"]._append(graph["_to"]))
n = V.size
volume = graph["_timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


# Chapter 1

## Preparing static graph

In [303]:
graph_static = [set() for _ in range(n + 1)]
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    graph_static[_from].add(_to)
    graph_static[_to].add(_from)


## Task 1.1

In [304]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
#print(E_count)

density = E_count * 2 / (n * (n - 1))
#print(density)

In [305]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_seen = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        V_seen.add(u)
        break
    while queue:
        u = queue.pop()
        u_adjacent_to_visit = graph_static[u].difference(V_seen)
        for v in u_adjacent_to_visit:
            V_seen.add(v)
            queue.append(v)
    V_to_visit = V_to_visit.difference(V_seen)
    connectivity_components.append(V_seen)

sizes = list(map(lambda x: len(x), connectivity_components))
max_component_size = max(sizes)
max_connectivity_component_index = sizes.index(max_component_size)
proportion = max_component_size / len(V)
#print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")


In [306]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component size = %i, max component proportion= % f"
      % (n, E_count, density, len(connectivity_components), max_component_size, proportion))


|V| = 1899, |E| = 13838, p = 0.007679, number of components = 4, max component size = 1893, max component proportion=  0.996840


## Task 1.2

In [337]:
component = list(connectivity_components[max_connectivity_component_index])
distances = [0] * (n + 2)
diameter = 0
radius = n + 1
distances_tmp = np.empty(n + 1, dtype=int)
for start in component:
    distances_tmp.fill(n + 1)
    V_visited = set()
    queue = [(start, 0)]
    queued = set([start])
    depth = 0
    u = start
    while queued:
        (u, depth) = queue.pop(0)
        queued.remove(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            distances_tmp[v] = min(distances_tmp[v], depth + 1)
            if v not in queued:
                queue.append((v, depth + 1))
                queued.add(v)
    for d in range(start, n + 1):
        distances[distances_tmp[d]] += 1
    diameter = max(diameter, depth)
    radius = min(radius, depth)
    if not start % 100:
        #print(start, "/", n)
        pass
#print(distances)
all_dist = 0
for i in range(diameter + 1):
    all_dist += distances[i]
print(all_dist, "=", max_component_size * (max_component_size - 1) // 2)


1790778 = 1790778


In [335]:
#percentile_90 = np.percentile(all_distances, 90)
percentile_90_ind = int(0.9 * all_dist)
percentile_90 = 0
ind_tmp = 0
for i in range(diameter + 1):
    ind_tmp += distances[i]
    if ind_tmp >= percentile_90_ind:
        percentile_90 = i
        break

In [355]:
def calculate_matrix(vertices):

    vertices_list = list(vertices)
    vertices_set = set(vertices)

    distances = []
    eccentricities = dict()
    distance_matrix = dict()
    for u in vertices_list:
        distance_matrix[u] = dict()
        for v in vertices_list:
            distance_matrix[u][v] = n + 1

    for start in vertices_list:
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        max_depth = 0
        while queue:
            u, depth = queue.pop(0)
            max_depth = max(max_depth, depth)
            queued.discard(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                if v in vertices_set:
                    distance = distance_matrix[start][v]
                    if depth + 1 < distance:
                        distance_matrix[start][v] = depth + 1
                        distance_matrix[v][start] = depth + 1
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
        eccentricities[start] = max_depth
    
    for u in vertices_list:
        for v in vertices_list:
            if u > v:
                distances.append(distance_matrix[u][v])

    return (distance_matrix, eccentricities, distances)


In [354]:
import random

component = list(connectivity_components[max_connectivity_component_index])

random_500_vertices = sorted(random.sample(component, 500))
random_1000_vertices = sorted(random.sample(component, 1000))

random_500_matrix, random_500_eccentricities, random_500_distances = calculate_matrix(
    random_500_vertices)
random_1000_matrix, random_1000_eccentricities, random_1000_distances = calculate_matrix(
    random_1000_vertices)

diameter_from_random_500 = max(random_500_eccentricities.values())
radius_from_random_500 = min(random_500_eccentricities.values())
percentile_90_from_random_500 = np.percentile(random_500_distances, 90)

print(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")

diameter_from_random_1000 = max(random_1000_eccentricities.values())
radius_from_random_1000 = min(random_1000_eccentricities.values())
percentile_90_from_random_1000 = np.percentile(random_1000_distances, 90)

print(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")

                    

{5: 6, 7: 6, 8: 5, 10: 6, 19: 5, 20: 5, 24: 6, 33: 5, 38: 5, 40: 5, 49: 6, 53: 5, 59: 5, 63: 5, 64: 5, 69: 5, 74: 5, 82: 5, 86: 5, 92: 6, 94: 5, 95: 5, 97: 5, 98: 5, 99: 5, 102: 5, 107: 5, 109: 5, 111: 6, 120: 5, 123: 5, 124: 6, 132: 5, 136: 6, 138: 6, 145: 5, 146: 5, 151: 5, 152: 5, 156: 6, 160: 6, 164: 5, 167: 6, 172: 5, 174: 5, 178: 6, 185: 5, 187: 7, 190: 5, 191: 5, 193: 5, 216: 6, 218: 6, 219: 5, 221: 5, 223: 6, 225: 6, 226: 5, 227: 6, 232: 5, 247: 6, 251: 5, 254: 5, 259: 5, 260: 5, 261: 6, 263: 5, 268: 6, 274: 5, 277: 5, 287: 6, 289: 5, 291: 5, 292: 6, 293: 6, 299: 5, 301: 6, 307: 5, 309: 6, 316: 5, 319: 5, 320: 6, 324: 5, 325: 5, 330: 6, 332: 5, 333: 5, 334: 5, 339: 5, 341: 5, 342: 5, 345: 5, 348: 6, 353: 5, 356: 5, 359: 6, 366: 6, 368: 5, 370: 6, 371: 5, 379: 5, 380: 6, 384: 6, 385: 5, 388: 5, 392: 5, 395: 6, 400: 5, 404: 5, 405: 5, 407: 5, 409: 5, 413: 5, 421: 6, 428: 6, 430: 5, 435: 5, 440: 5, 444: 6, 453: 5, 456: 4, 457: 5, 459: 5, 461: 6, 462: 5, 469: 5, 470: 6, 471: 6, 479

In [348]:
def snowball(limit):
    vertices = {component[0], component[1]}
    while len(vertices) < limit:
        for v in vertices:
            if len(vertices) < limit:
                vertices = vertices.union(graph_static[v])
    return sorted(list(vertices))

snowball_500_vertices = snowball(500)
snowball_1000_vertices = snowball(1000)

snowball_500_matrix, snowball_500_eccentricities, snowball_500_distances = calculate_matrix(
    snowball_500_vertices)
snowball_1000_matrix, snowball_1000_eccentricities, snowball_1000_distances = calculate_matrix(
    snowball_1000_vertices)

diameter_from_snowball_500 = max(snowball_500_eccentricities.values())
radius_from_snowball_500 = min(snowball_500_eccentricities.values())
percentile_90_from_snowball_500 = np.percentile(snowball_500_distances, 90)

print(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")

diameter_from_snowball_1000 = max(snowball_1000_eccentricities.values())
radius_from_snowball_1000 = min(snowball_1000_eccentricities.values())
percentile_90_from_snowball_1000 = np.percentile(snowball_1000_distances, 90)

print(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")


diameter_from_snowball_500 = 1807, radius_from_snowball_500 = 1, percentile_90_from_snowball_500 = 3.0
diameter_from_snowball_1000 = 1899, radius_from_snowball_1000 = 1, percentile_90_from_snowball_1000 = 3.0


In [336]:
# 1.2
print("diameter = %i, raduis = %i, percentile_90 = %i" 
      % (diameter, radius, percentile_90))

diameter = 8, raduis = 4, percentile_90 = 4


## Task 1.3

In [ ]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [ ]:
#1.3
print("Cl_average = %f" % (Cl_average))

Cl_average = 0.096853


## Task 1.4

In [ ]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for i in range(1, n + 1):
    ki = len(graph_static[i])
    R1 += ki
    R2 += ki**2
    R3 += ki**3
    for j in graph_static[i]:
        kj = len(graph_static[j])
        Re += ki * kj
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [ ]:
print("Degree associativity = %f" % (degree_associativity))

Degree associativity = -0.055710
